In [14]:
from textprepper.modifiers import LowerText, SpellChecking
from textprepper import DocumentPreprocessorPipe

from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import chain, RunnableLambda, RunnablePassthrough

# How to use a preprocessor in a LCEL chain

- Use case: If you want to manipulate the query of the user like checking the spelling beforehand

In [53]:
dummy_example_instruct = """Answer the question
Question: {question}
"""
dummy_prompt = ChatPromptTemplate.from_template(dummy_example_instruct)

## Options to build LCEL's Chains

Please keep in mind:

<b>The Input of a preprocessor must be a langchain Document or a string!</b>
Otherwise it will raise a ValueError.

#### 1. Option Direct

In [64]:
dummy_question = "This is a simple misteke" # mistEke
lcel_chain = RunnablePassthrough() | SpellChecking() | dummy_prompt 
print(lcel_chain.invoke(dummy_question).messages[0].content)

Answer the question
Question: This is a simple mistake



#### 2. Option with RunnableLambda
- It's the same like option 1, because it's not the first item in the chain

In [65]:
lower_lambda = RunnableLambda(SpellChecking())
lcel_chain =  RunnablePassthrough() | lower_lambda | dummy_prompt
print(lcel_chain.invoke(dummy_question).messages[0].content)

Answer the question
Question: This is a simple mistake



#### 3. Option with chain decorator 

- Helpful if you're wrapped your input/intermediate outputs in a dictionary 

In [66]:
@chain
def processor(input: dict):
    text = input["question"]
    return SpellChecking()(text)

In [67]:
lcel_chain = {"question": RunnablePassthrough()} | processor | dummy_prompt
print(lcel_chain.invoke(dummy_question).messages[0].content)

Answer the question
Question: This is a simple mistake



### 4. Option with DocumentPipe

In [63]:
pipe = DocumentPreprocessorPipe([LowerText(), SpellChecking()])
lcel_chain = RunnablePassthrough() | pipe | dummy_prompt 
print(lcel_chain.invoke(dummy_question).messages[0].content)

Answer the question
Question: this is a simple mistake

